In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import pandas as pd
import tensorflow as tf

2024-02-29 19:27:30.323687: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 19:27:30.323793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 19:27:30.600004: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# import os
# os.listdir('/kaggle/input/fra-eng/fra.txt')

In [3]:
# '/kaggle/input/fra-eng/fra.txt'
# '/kaggle/input/parallel_data.txt'
with open('/kaggle/input/parallel_data.txt') as f:
    lines = f.read().split("\n")[:-1]
    print(len(lines))
text_pairs = []
for line in lines:
    nep, eng = line.split("\t")
    eng = 'START_ '+eng+' _END'
    text_pairs.append((nep, eng))

34374


In [4]:
df = pd.DataFrame(data=text_pairs, columns=['NEP', 'ENG'])

In [5]:
df

,NEP,ENG
0,धृतराष्ट्र उवाच धर्मक्षेत्रे कुरुक्षेत्रे समवे...,START_ START_ dhrtarastra said o sanjaya what ...
1,सञ्जय उवाच दृष्ट्वा तु पाण्डवानीकं व्यूढं दुर्...,START_ START_ sanjaya said seeing the army of ...
2,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम्। व्...,START_ START_ o teacher behold this mighty arm...
3,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि। युयुधान...,START_ START_ the heroes and mighty archers co...
4,धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान्। पुरु...,START_ START_ dhrstaketu cekitana and the valo...
...,...,...
34369,अगछःसोमिनो गर्हम,START_ START_ and sought the somapourers home ...
34370,तवं तयमिन्द्र मर्त्यमास्त्रबुध्नाय वेन्यम,START_ START_ venya that mortal man hast thou ...
34371,मुहुःश्रथ्ना मनस्यवे,START_ START_ o indra many a time set free _EN...
34372,तवं तयमिन्द्र सूर्यं पश्चा सन्तं पुरस कर्धि,START_ START_ bring indra to the east again th...


In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    words = text.split()
    filtered_text = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_text)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df.ENG = df.ENG.apply(remove_stopwords)

In [8]:
# Lowercase all characters
df.ENG = df.ENG.apply(lambda x: re.sub(' _END','',re.sub('START_ ','',x)))
df.NEP=df.NEP.apply(lambda x: x.lower())
df.ENG=df.ENG.apply(lambda x: x.lower())

In [9]:
df

,NEP,ENG
0,धृतराष्ट्र उवाच धर्मक्षेत्रे कुरुक्षेत्रे समवे...,dhrtarastra said sanjaya men sons pandu kuruks...
1,सञ्जय उवाच दृष्ट्वा तु पाण्डवानीकं व्यूढं दुर्...,sanjaya said seeing army sons pandu marshalled...
2,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम्। व्...,teacher behold mighty army sons pandu marshall...
3,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि। युयुधान...,heroes mighty archers comparable war bhima arj...
4,धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान्। पुरु...,dhrstaketu cekitana valourous king kasi kuntib...
...,...,...
34369,अगछःसोमिनो गर्हम,sought somapourers home
34370,तवं तयमिन्द्र मर्त्यमास्त्रबुध्नाय वेन्यम,venya mortal man hast thou āstrabudhna devout
34371,मुहुःश्रथ्ना मनस्यवे,indra many time set free
34372,तवं तयमिन्द्र सूर्यं पश्चा सन्तं पुरस कर्धि,bring indra east sun west


In [10]:
# Remove quotes
df.NEP=df.NEP.apply(lambda x: re.sub("'", '', x))
df.ENG=df.ENG.apply(lambda x: re.sub("'", '', x))

In [11]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
df.NEP=df.NEP.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df.ENG=df.ENG.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [12]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
df.NEP=df.NEP.apply(lambda x: x.translate(remove_digits))

df.NEP = df.NEP.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))
df.ENG = df.ENG.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [13]:
# Remove extra spaces
df.NEP=df.NEP.apply(lambda x: x.strip())
df.ENG=df.ENG.apply(lambda x: x.strip())
df.NEP=df.NEP.apply(lambda x: re.sub(" +", " ", x))
df.ENG=df.ENG.apply(lambda x: re.sub(" +", " ", x))

In [14]:
df.ENG = df.ENG.apply(lambda x : 'START_ '+ x + ' _END')

In [15]:
df

,NEP,ENG
0,धृतराष्ट्र उवाच धर्मक्षेत्रे कुरुक्षेत्रे समवे...,START_ dhrtarastra said sanjaya men sons pandu...
1,सञ्जय उवाच दृष्ट्वा तु पाण्डवानीकं व्यूढं दुर्...,START_ sanjaya said seeing army sons pandu mar...
2,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम्। व्...,START_ teacher behold mighty army sons pandu m...
3,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि। युयुधान...,START_ heroes mighty archers comparable war bh...
4,धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान्। पुरु...,START_ dhrstaketu cekitana valourous king kasi...
...,...,...
34369,अगछःसोमिनो गर्हम,START_ sought somapourers home _END
34370,तवं तयमिन्द्र मर्त्यमास्त्रबुध्नाय वेन्यम,START_ venya mortal man hast thou āstrabudhna ...
34371,मुहुःश्रथ्ना मनस्यवे,START_ indra many time set free _END
34372,तवं तयमिन्द्र सूर्यं पश्चा सन्तं पुरस कर्धि,START_ bring indra east sun west _END


In [16]:
# Vocabulary of Nepali 
all_nepali_words=set()
for nep in df.NEP:
    for word in nep.split():
        if word not in all_nepali_words:
            all_nepali_words.add(word)
# Vocabulary of English
all_eng_words=set()
for eng in df.ENG:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [17]:
len(all_nepali_words), len(all_eng_words)

(125544, 24245)

In [18]:
all_nepali_words.add('<UNK>')
all_eng_words.add('<UNK>')

In [19]:
# Max Length of source sequence
lenght_list=[]
for l in df.NEP:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

78

In [20]:
# Max Length of target sequence
lenght_list=[]
for l in df.ENG:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

117

In [21]:
input_words = sorted(list(all_nepali_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_nepali_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(125545, 24246)

In [22]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

24247

In [23]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [24]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [25]:
df = shuffle(df)
df.head(5)

,NEP,ENG
17172,अहं पुत्रसहाया त्वामुपासे गतचेतसम्। सिंहेन निह...,START_ like cow followed calf would attend upo...
32438,ईळेन्यं परथमं मातरिश्वा देवास्ततक्षुर्मनवे यजत्रम,START_ mātariśvan gods fashioned holy man firs...
25664,आ हि सथाथो दिविस्प्र्शम,START_ sacrifice reaches heaven _END
9357,राजमूलो हि धर्मश्च जयश्च जयतां वर। तस्मात्सर्व...,START_ ravana best among victorious indeed kin...
6416,यासां नाददते शुल्कं ज्ञातयो न स विक्रयः ।अर्हण...,START_ relatives appropriate use gratuity give...


In [26]:
# Train - Test Split
X, y = df.NEP, df.ENG
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((30936,), (3438,))

In [27]:
X_train

13307    ततो निविष्टां ध्वजिनीं गङ्गामन्वाश्रितां नदीम्...
31156    आ यस्तस्थौ भुवनान्यमर्त्यो विश्वानि सोमः परि त...
12339    दीर्घबाहुर्नरव्याघ्रो जटिलत्वमधारयत्। तौ तदा च...
24225    अग्ने तोकस्य नस्तने तनूनामप्रयुछन दीद्यद बोधि ...
12116    सभार्यं सम्प्रसुप्तं तं भ्रातरं वीक्ष्य लक्ष्म...
                               ...                        
23737    उत वा यः सहस्य परविद्वान मर्तो मर्तं मर्चयति द...
26332    तद अस्तु मित्रावरुणा तद अग्ने शं योर अस्मभ्यम ...
5772     किञ्च ये प्रत्येष्यन्ति तैरीदृग् आश्चर्य्यं कर...
4452     तदानीं व्यवस्थापकानाम् एका यीशुमवदत् हे उपदेशक...
26721             पुरू चिद अस्मयुस तिर आङगूषो मर्त्येष्व आ
Name: NEP, Length: 30936, dtype: object

In [28]:
def generate_batch(X = X_train, y = y_train, batch_size = 32):
    ''' Generate a batch of data '''    
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
#                 input_text = input_text.decode('utf-8')
#                 target_text = target_text.decode('utf-8')
#                 print(input_text)
                for t, word in enumerate(input_text.split()):
                    if word in input_token_index:
                        encoder_input_data[i, t] = input_token_index[word]  # encoder input seq
                    else:
                        # Handle out-of-vocabulary words here
                        # For example, you could assign a special index for unknown words
                        encoder_input_data[i, t] = input_token_index['<UNK>']
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        if word in target_token_index:
                            decoder_input_data[i, t] = target_token_index[word]  # decoder input seq
                        else:
                            decoder_input_data[i, t] = target_token_index['<UNK>']

                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        if word in target_token_index:
                            decoder_target_data[i, t - 1, target_token_index[word]] = 1.
                        else:
                            # Handle out-of-vocabulary words here
                            pass  # You may choose to ignore or handle them differently            
            yield((encoder_input_data, decoder_input_data), decoder_target_data)

In [29]:
latent_dim = 50

In [30]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [31]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model((encoder_inputs, decoder_inputs), decoder_outputs)

In [32]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [33]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 45

In [34]:
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/45
966/966 ━━━━━━━━━━━━━━━━━━━━ 490s 501ms/step - acc: 0.0084 - loss: 8.5791 - val_acc: 0.0085 - val_loss: 7.8615
Epoch 2/45
966/966 ━━━━━━━━━━━━━━━━━━━━ 480s 497ms/step - acc: 0.0085 - loss: 7.8660 - val_acc: 0.0085 - val_loss: 7.8548
Epoch 3/45
966/966 ━━━━━━━━━━━━━━━━━━━━ 478s 495ms/step - acc: 0.0085 - loss: 7.8621 - val_acc: 0.0085 - val_loss: 7.8542
Epoch 4/45
966/966 ━━━━━━━━━━━━━━━━━━━━ 478s 495ms/step - acc: 0.0085 - loss: 7.8612 - val_acc: 0.0085 - val_loss: 7.8536
Epoch 5/45
966/966 ━━━━━━━━━━━━━━━━━━━━ 478s 495ms/step - acc: 0.0085 - loss: 7.8535 - val_acc: 0.0085 - val_loss: 7.8249
Epoch 6/45
966/966 ━━━━━━━━━━━━━━━━━━━━ 483s 500ms/step - acc: 0.0085 - loss: 7.8293 - val_acc: 0.0085 - val_loss: 7.8064
Epoch 7/45
966/966 ━━━━━━━━━━━━━━━━━━━━ 483s 500ms/step - acc: 0.0085 - loss: 7.8106 - val_acc: 0.0085 - val_loss: 7.7928
Epoch 8/45
966/966 ━━━━━━━━━━━━━━━━━━━━ 480s 496ms/step - acc: 0.0085 - loss: 7.7976 - val_acc: 0.0087 - val_loss: 7.7801
Epoch 9/45
966/966 ━━━━━

In [35]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [36]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [37]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [38]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input Nepali sentence: ततो निविष्टां ध्वजिनीं गङ्गामन्वाश्रितां नदीम्। निषादराजो दृष्ट्वैव ज्ञातीन्सन्त्वरितोऽऽब्रवीत्।।
Actual English Translation:  guha king nishadas observed bannered army encamped along bank river ganga hurried said kinsmen 
Predicted English Translation:  rama rama rama rama 


In [39]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Input Nepali sentence: आ यस्तस्थौ भुवनान्यमर्त्यो विश्वानि सोमः परि तान्यर्षति
Actual English Translation:  hath come anear creatures life immortal soma flows onward 
Predicted English Translation:  may may may us 


In [40]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Input Nepali sentence: दीर्घबाहुर्नरव्याघ्रो जटिलत्वमधारयत्। तौ तदा चीरवसनौ जटामण्डलधारिणौ। आशोभेतामृषिसमौ भ्रातरौ रामलक्ष्मणौ।।
Actual English Translation:  longarmed rama best men put locks matted hair two brothers rama lakshmana stood dressed bark wearing crowns matted hair resembled two resplendent ascetics 
Predicted English Translation:  rama rama rama rama rama rama rama rama 


In [41]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input Nepali sentence: अग्ने तोकस्य नस्तने तनूनामप्रयुछन दीद्यद बोधि गोपाः
Actual English Translation:  refulgent everheedful keeper agni us seed offspring 
Predicted English Translation:  may thou us us 


In [42]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input Nepali sentence: सभार्यं सम्प्रसुप्तं तं भ्रातरं वीक्ष्य लक्ष्मणः। कथयामास सूताय रामस्य विविधान् गुणान्।।
Actual English Translation:  seen brother along wife fallen asleep lakshmana narrated many virtues rama sumantra 
Predicted English Translation:  rama rama rama rama rama 


In [43]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input Nepali sentence: ततः पश्चान्महातेजा नाराचान्भास्करोपमान्। जिघांसू राक्षसङ्कृद्धस्त्रयोदश समाददे।।
Actual English Translation:  enraged effulgent rama took thirteen darts radiant like sun wish kill demon 
Predicted English Translation:  rama rama rama rama shall 


In [44]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input Nepali sentence: तत्रापश्यत्स मेघाभं न्यग्रोधमृषिभिर्वृतम्। समन्ताद्यस्य ताश्शाखाश्शतयोजनमायताः।।
Actual English Translation:  saw banyan tree branches spread sides measuring hundred yojanas looking like cloud providing shelter sages 
Predicted English Translation:  rama rama rama 


In [45]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input Nepali sentence: तां तु दृष्ट्वा महाबाहो दारितां च स्तनान्तरे। आशीविष इव क्रुद्धो निश्वसन्नभ्यभाषथाः।।
Actual English Translation:  seeing blood flowing breasts became furious like hissing serpent said 
Predicted English Translation:  rama rama rama 


In [46]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input Nepali sentence: तदानीं राजा तान् प्रतिवदिष्यति युष्मानहं सत्यं वदामि ममैतेषां भ्रातृणां मध्ये कञ्चनैकं क्षुद्रतमं प्रति यद् अकुरुत तन्मां प्रत्यकुरुत।
Actual English Translation:  king shall answer say unto verily say unto inasmuch ye done unto one least brethren ye done unto 
Predicted English Translation:  rama rama rama rama rama rama rama rama 


In [47]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input Nepali sentence: इन्द्राग्नी शर्म यछतम
Actual English Translation:  indra agni send us bliss 
Predicted English Translation:  may us us 


In [48]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Input Nepali sentence: किन्तु यीशुस्तां किमपि नोक्तवान् ततः शिष्या आगत्य तं निवेदयामासुः एषा योषिद् अस्माकं पश्चाद् उच्चैराहूयागच्छति एनां विसृजतु।
Actual English Translation:  answered word disciples came besought saying send away crieth us 
Predicted English Translation:  rama rama rama rama rama 


In [49]:
# VALIDATION SET

In [50]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [51]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Input Nepali sentence: ततो निविष्टां ध्वजिनीं गङ्गामन्वाश्रितां नदीम्। निषादराजो दृष्ट्वैव ज्ञातीन्सन्त्वरितोऽऽब्रवीत्।।
Actual English Translation:  guha king nishadas observed bannered army encamped along bank river ganga hurried said kinsmen 
Predicted English Translation:  may may thou us us us us 


In [52]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input Nepali sentence: आ यस्तस्थौ भुवनान्यमर्त्यो विश्वानि सोमः परि तान्यर्षति
Actual English Translation:  hath come anear creatures life immortal soma flows onward 
Predicted English Translation:  rama rama rama rama 


In [53]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input Nepali sentence: दीर्घबाहुर्नरव्याघ्रो जटिलत्वमधारयत्। तौ तदा चीरवसनौ जटामण्डलधारिणौ। आशोभेतामृषिसमौ भ्रातरौ रामलक्ष्मणौ।।
Actual English Translation:  longarmed rama best men put locks matted hair two brothers rama lakshmana stood dressed bark wearing crowns matted hair resembled two resplendent ascetics 
Predicted English Translation:  rama rama rama rama 


In [54]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input Nepali sentence: अग्ने तोकस्य नस्तने तनूनामप्रयुछन दीद्यद बोधि गोपाः
Actual English Translation:  refulgent everheedful keeper agni us seed offspring 
Predicted English Translation:  may us us 


In [55]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Input Nepali sentence: सभार्यं सम्प्रसुप्तं तं भ्रातरं वीक्ष्य लक्ष्मणः। कथयामास सूताय रामस्य विविधान् गुणान्।।
Actual English Translation:  seen brother along wife fallen asleep lakshmana narrated many virtues rama sumantra 
Predicted English Translation:  rama rama rama rama shall 


In [56]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input Nepali sentence: ततः पश्चान्महातेजा नाराचान्भास्करोपमान्। जिघांसू राक्षसङ्कृद्धस्त्रयोदश समाददे।।
Actual English Translation:  enraged effulgent rama took thirteen darts radiant like sun wish kill demon 
Predicted English Translation:  rama rama rama rama 


In [57]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Input Nepali sentence: तत्रापश्यत्स मेघाभं न्यग्रोधमृषिभिर्वृतम्। समन्ताद्यस्य ताश्शाखाश्शतयोजनमायताः।।
Actual English Translation:  saw banyan tree branches spread sides measuring hundred yojanas looking like cloud providing shelter sages 
Predicted English Translation:  rama rama rama 


In [58]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input Nepali sentence: तां तु दृष्ट्वा महाबाहो दारितां च स्तनान्तरे। आशीविष इव क्रुद्धो निश्वसन्नभ्यभाषथाः।।
Actual English Translation:  seeing blood flowing breasts became furious like hissing serpent said 
Predicted English Translation:  rama rama rama 


In [59]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input Nepali sentence: तदानीं राजा तान् प्रतिवदिष्यति युष्मानहं सत्यं वदामि ममैतेषां भ्रातृणां मध्ये कञ्चनैकं क्षुद्रतमं प्रति यद् अकुरुत तन्मां प्रत्यकुरुत।
Actual English Translation:  king shall answer say unto verily say unto inasmuch ye done unto one least brethren ye done unto 
Predicted English Translation:  rama rama rama rama rama 


In [60]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input Nepali sentence: इन्द्राग्नी शर्म यछतम
Actual English Translation:  indra agni send us bliss 
Predicted English Translation:  may thou us us us 


In [61]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Input Nepali sentence: किन्तु यीशुस्तां किमपि नोक्तवान् ततः शिष्या आगत्य तं निवेदयामासुः एषा योषिद् अस्माकं पश्चाद् उच्चैराहूयागच्छति एनां विसृजतु।
Actual English Translation:  answered word disciples came besought saying send away crieth us 
Predicted English Translation:  rama rama rama rama 


In [62]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input Nepali sentence: चारुप्रतीकाहुतः
Actual English Translation:  worshipped fair look upon 
Predicted English Translation:  may thou thou thou us us us us 


In [63]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input Nepali sentence: रामशोकमहाभोगस्सीताविरहपारगः। श्वसितोर्मि महावर्तो बाष्पफेनजालाविलः। बाहुविक्षेपमीनौघो विक्रन्दित महास्वनः। प्रकीर्णकेशशैवालः कैकेयीबडबामुखः। ममाश्रुवेगप्रभवः कुब्जावाक्यमहाग्रहः। वरवेलो नृशंसाया रामप्रव्राजनायतः। यस्मिन्बत निमग्नोऽहं कौसल्ये राघवं विना। दुस्तरो जीवता देवि मयाऽयं शोकसागरः।।
Actual English Translation:  queen kausalya completely immersed ocean sorrow misfortune due ramas separation breadth sitas separation end shore sighs turbulent waves whirlpools tears foam turbid waters waving arms fishes cries agony roars dishevelled hair moss kaikeyi mouth badaba copious tears source words hunchback manthara monstrous crocodiles cruel boons kaikeyi shores without rama cannot cross sea sorrow alive ah pity 
Predicted English Translation:  rama rama rama 


In [64]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Input Nepali sentence: वसिष्ठस्तु तदा राममुक्त्वा राजपुरोहितः। अब्रवीद्धर्मसंयुक्तं पुनरेवापरं वचः।।
Actual English Translation:  vasistha family priest continued speak rama righteousness 
Predicted English Translation:  thou us us us us 


In [65]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Nepali sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input Nepali sentence: अपास्मात परेयान न तदोको अस्ति पर्णन्तमन्यमरणं चिदिचःेत
Actual English Translation:  let departno home rest rather seek stranger support 
Predicted English Translation:  may thou us us us 
